In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))
    !nvidia-smi

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4
Tue Jun 28 20:41:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                            

In [3]:
! pip install datasets transformers[sentencepiece] sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 362 kB 32.4 MB/s 
     |████████████████████████████████| 4.4 MB 69.6 MB/s 
     |████████████████████████████████| 92 kB 13.4 MB/s 
     |████████████████████████████████| 1.1 MB 71.8 MB/s 
     |████████████████████████████████| 140 kB 102.3 MB/s 
     |████████████████████████████████| 212 kB 64.8 MB/s 
     |████████████████████████████████| 101 kB 12.6 MB/s 
     |████████████████████████████████| 596 kB 69.5 MB/s 
     |████████████████████████████████| 127 kB 95.7 MB/s 
     |████████████████████████████████| 144 kB 80.7 MB/s 
     |████████████████████████████████| 94 kB 4.7 MB/s 
     |████████████████████████████████| 271 kB 73.5 MB/s 
     |████████████████████████████████| 6.6 MB 4.8 MB/s 
     |████████████████████████████████| 1.2 MB 53.4 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstal

In [4]:
import transformers

print(transformers.__version__)

4.20.1


# Fine-tuning a model on a normalization task



In [5]:
model_checkpoint = "moussaKam/AraBART"

## Loading the dataset

In [6]:
!pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 43 kB 44 kB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=5723b1b3612a76108031802494ce6cb46a20db0d4c0cdeca9bda6b420df5b3b1
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


In [7]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset('csv', data_files={'train': ['/content/drive/MyDrive/Academic_Papers/SPIRAL/Space_issues/train.csv'],'test': '/content/drive/MyDrive/Academic_Papers/SPIRAL/Space_issues/test.csv'})
metric = load_metric("sacrebleu")

Using custom data configuration default-6fdb8b7f00d26e28


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-6fdb8b7f00d26e28/0.0.0/51cce309a08df9c4d82ffd9363bbe090bf173197fc01a71b034e8594995a1a58. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['index', 'word', 'Norm'],
        num_rows: 1716190
    })
    test: Dataset({
        features: ['index', 'word', 'Norm'],
        num_rows: 429047
    })
})

In [9]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

To access an actual element, you need to select a split first, then give an index:

In [10]:
raw_datasets["train"][0]

{'Norm': 'بمزاولة', 'index': 63220, 'word': 'ب مزاولة'}

To get a sense of what the data looks like, the following function will show some examples picked randomly in the dataset.

In [11]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [12]:
show_random_elements(raw_datasets["train"])

,index,word,Norm
0,82034,والم رتدّون,والمرتدّون
1,42491,أد اؤه,أداؤه
2,319467,نظر تنّ,نظرتنّ
3,74410,أ نتنت,أنتنت
4,202454,أ ضا,أضا


In [13]:
metric

Metric(name: "sacrebleu", features: {'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')}, usage: """
Produces BLEU scores along with its sufficient statistics
from a source against one or more references.

Args:
    predictions (`list` of `str`): list of translations to score. Each translation should be tokenized into a list of tokens.
    references (`list` of `list` of `str`): A list of lists of references. The contents of the first sub-list are the references for the first prediction, the contents of the second sub-list are for the second prediction, etc. Note that there must be the same number of references for each prediction (i.e. all sub-lists must be of the same length).
    smooth_method (`str`): The smoothing method to use, defaults to `'exp'`. Possible values are:
        - `'none'`: no smoothing
        - `'floor'`: increment zero counts
        - `'add-k'`: increment num/deno

## Preprocessing the data

In [14]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

In [15]:
prefix = ""

In [16]:
max_input_length = 16
max_target_length = 16

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["word"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["Norm"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[0, 35, 26554, 2], [0, 25214, 502, 112, 2]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1]], 'labels': [[0, 690, 34817, 13, 2], [0, 25214, 1399, 2]]}

In [18]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Parameter 'function'=<function preprocess_function at 0x7ff9c483e170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/1717 [00:00<?, ?ba/s]

  0%|          | 0/430 [00:00<?, ?ba/s]

## Fine-tuning the model

Now that our data is ready, we can download the pretrained model and fine-tune it. Since our task is of the sequence-to-sequence kind, we use the `AutoModelForSeq2SeqLM` class. Like with the tokenizer, the `from_pretrained` method will download and cache the model for us.

In [19]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/531M [00:00<?, ?B/s]

In [20]:
source_lang='word'
target_lang='norm'

In [21]:
batch_size = 64
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=15,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False,
)

In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score
from sklearn.metrics import precision_recall_fscore_support

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)


    result1= accuracy_score(decoded_labels, decoded_preds)
    results2= f1_score(decoded_labels, decoded_preds, average='macro')
    results3= precision_recall_fscore_support(decoded_labels, decoded_preds, average='macro')
    
    return {
        "f1-score": results2,
        "accuracy": result1,
        "All": results3,
    }

Then we just need to pass all of this along with our datasets to the `Seq2SeqTrainer`:

In [24]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Using cuda_amp half precision backend


We can now finetune our model by just calling the `train` method:

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: Norm, index, word. If Norm, index, word are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 1716190
  Num Epochs = 15
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 402240


Epoch,Training Loss,Validation Loss


Saving model checkpoint to AraBART-finetuned-word-to-norm/checkpoint-500
Configuration saved in AraBART-finetuned-word-to-norm/checkpoint-500/config.json
Model weights saved in AraBART-finetuned-word-to-norm/checkpoint-500/pytorch_model.bin
tokenizer config file saved in AraBART-finetuned-word-to-norm/checkpoint-500/tokenizer_config.json
Special tokens file saved in AraBART-finetuned-word-to-norm/checkpoint-500/special_tokens_map.json
Saving model checkpoint to AraBART-finetuned-word-to-norm/checkpoint-1000
Configuration saved in AraBART-finetuned-word-to-norm/checkpoint-1000/config.json
Model weights saved in AraBART-finetuned-word-to-norm/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in AraBART-finetuned-word-to-norm/checkpoint-1000/tokenizer_config.json
Special tokens file saved in AraBART-finetuned-word-to-norm/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to AraBART-finetuned-word-to-norm/checkpoint-1500
Configuration saved in AraBART-finetuned-wo

Epoch,Training Loss,Validation Loss,F1-score,Accuracy,All
1,0.228000,0.158945,0.790217,0.879955,"(0.8048262400993602, 0.7932012922999004, 0.7902166867298706, None)"
2,0.135200,0.098166,0.830669,0.904761,"(0.8417004437990038, 0.8342557925533596, 0.8306691025440553, None)"
3,0.095300,0.070622,0.859958,0.922631,"(0.8695333059147937, 0.8626029449425986, 0.85995836901009, None)"
4,0.070200,0.056393,0.851714,0.917133,"(0.8613432447905102, 0.8551060971807736, 0.8517135022078148, None)"
5,0.053900,0.046937,0.850235,0.915995,"(0.8597441677808924, 0.8538792431801696, 0.8502348251523739, None)"


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "(0.8048262400993602, 0.7932012922999004, 0.7902166867298706, None)" of type <class 'tuple'> for key "eval/All" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Saving model checkpoint to AraBART-finetuned-word-to-norm/checkpoint-27000
Configuration saved in AraBART-finetuned-word-to-n

You can now share this model with all your friends, family, favorite pets: they can all load it with the identifier `"your-username/the-name-you-picked"` so for instance:

```python
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("sgugger/my-awesome-model")
```

In [ ]:
trainer.evaluate()

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
import numpy as np

preds = np.argmax(predictions.predictions, axis=-1)

In [ ]:
predictions.predictions

In [ ]:
predictions.label_ids

In [ ]:
decoded_preds = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Replace -100 in the labels as we can't decode them.
labels = np.where(predictions.label_ids != -100, predictions.label_ids, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

# Some simple post-processing
decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
f1_score(decoded_labels, decoded_preds, average='macro')
accuracy_score(decoded_labels, decoded_preds)